# 準備

## artのインストール

In [1]:
!pip3 install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 30.3 MB/s 
     |████████████████████████████████| 26.3 MB 80 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


## ライブラリのインポート

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()
from keras import backend as K

# ART
import art
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

## CIFAR10のロード・前処理

In [ ]:
# CIFAR10のロード。
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# CIFAR10のラベル。
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

In [ ]:
# 正規化。
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# ラベルをOne-hot-vector化。
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Classifier(k-WTAなし)の作成



## モデル定義

In [ ]:
# モデルの定義。
inputs = Input(shape=(32, 32, 3))
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model = Model(inputs, y)

# モデルのコンパイル。
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584

## 学習の実行

In [ ]:
# 学習の実行。
model.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30


2021-10-30 19:29:35.489207: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


50000/50000 [==============================] - ETA: 0s - loss: 2.1135 - accuracy: 0.1905

/Users/harada/opt/anaconda3/envs/k-wta/lib/python3.7/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


50000/50000 [==============================] - 229s 5ms/sample - loss: 2.1135 - accuracy: 0.1905 - val_loss: 1.9977 - val_accuracy: 0.2366
Epoch 2/30
50000/50000 [==============================] - 228s 5ms/sample - loss: 1.7288 - accuracy: 0.3385 - val_loss: 1.6426 - val_accuracy: 0.3998
Epoch 3/30
50000/50000 [==============================] - 227s 5ms/sample - loss: 1.4364 - accuracy: 0.4611 - val_loss: 1.3250 - val_accuracy: 0.5234
Epoch 4/30
50000/50000 [==============================] - 229s 5ms/sample - loss: 1.2325 - accuracy: 0.5443 - val_loss: 1.2154 - val_accuracy: 0.5732
Epoch 5/30
50000/50000 [==============================] - 227s 5ms/sample - loss: 1.0705 - accuracy: 0.6081 - val_loss: 1.0749 - val_accuracy: 0.6268
Epoch 6/30
50000/50000 [==============================] - 229s 5ms/sample - loss: 0.9750 - accuracy: 0.6467 - val_loss: 0.9981 - val_accuracy: 0.6493
Epoch 7/30
50000/50000 [==============================] - 227s 5ms/sample - loss: 0.8681 - accuracy: 0.6883 - v

## モデルの精度評価

In [ ]:
model.evaluate(X_test, y_test)

[0.5525944200992584, 0.8335]

# Classifier(k-WTAあり)の作成

## k-WTAの作成

In [ ]:
class KWTA(tf.keras.layers.Layer):
  def __init__(self, k, **kwargs) :
    super().__init__(**kwargs)
    self.k = k

  def call(self, x):
    topk = tf.math.top_k(x[0], k=self.k)   # 上位k個の値を抽出
    topk_min = K.min(topk.values)         # topkの最小値を抽出
    comp = tf.dtypes.cast(x >= topk_min, tf.float32)   # topkの最小値以上の値の部分を1, より小さい値の部分を0にしたTensorを生成
    return tf.math.multiply(x,comp)        # x * compより、topkの最小値以上の値のみそのまま、他は0になるようにする

## モデル定義

In [ ]:
# モデルの定義。
inputs = Input(shape=(32, 32, 3))
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024)(x)
x = KWTA(512)(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model_kwta = Model(inputs, y)

# モデルのコンパイル。
model_kwta.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_kwta.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 128)       1475

## 学習の実行

In [ ]:
# 学習の実行。
model_kwta.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 223s 4ms/sample - loss: 2.0193 - accuracy: 0.2127 - val_loss: 1.8150 - val_accuracy: 0.3091
Epoch 2/30
50000/50000 [==============================] - 225s 5ms/sample - loss: 1.6166 - accuracy: 0.3884 - val_loss: 1.5469 - val_accuracy: 0.4386
Epoch 3/30
50000/50000 [==============================] - 224s 4ms/sample - loss: 1.3662 - accuracy: 0.4935 - val_loss: 1.3358 - val_accuracy: 0.5141
Epoch 4/30
50000/50000 [==============================] - 223s 4ms/sample - loss: 1.1790 - accuracy: 0.5674 - val_loss: 1.1348 - val_accuracy: 0.6002
Epoch 5/30
50000/50000 [==============================] - 224s 4ms/sample - loss: 1.0334 - accuracy: 0.6262 - val_loss: 0.9981 - val_accuracy: 0.6602
Epoch 6/30
50000/50000 [==============================] - 224s 4ms/sample - loss: 0.9198 - accuracy: 0.6693 - val_loss: 0.9807 - val_accuracy: 0.6505
Epoch 7/30
50000/50000 [==========================

## モデルの精度評価

In [ ]:
model_kwta.evaluate(X_test, y_test)

[0.5696513920068741, 0.825]

# 敵対的攻撃

## 非標的型攻撃としてのFGSMによる摂動を作成
以下、この摂動を上乗せした画像のことを「敵対的画像」と呼ぶ。

In [ ]:
# 入力データの特徴量の最小値・最大値を指定。
# 特徴量は0.0～1.0の範囲に収まるように正規化しているため、最小値は0.0、最大値は1.0とする。
min_pixel_value = 0.0
max_pixel_value = 1.0

### k-WTAなし

In [ ]:
# モデルをART Keras Classifierでラップ。
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
# FGSMインスタンスの作成。
attack = FastGradientMethod(estimator=classifier, eps=0.10, targeted=False)
# 敵対的サンプルの生成（ベース画像はテストデータとする）。
X_adv = attack.generate(x=X_test)

### k-WTAあり

In [ ]:
classifier_kwta = KerasClassifier(model=model_kwta, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
attack_kwta = FastGradientMethod(estimator=classifier_kwta, eps=0.10, targeted=False)
X_adv_kwta = attack_kwta.generate(X_test)

## 敵対的画像に対するモデルの精度評価

### k-WTAなし

In [ ]:
model.evaluate(X_adv, y_test)

[6.526164630889893, 0.1024]

### k-WTAあり

In [ ]:
model_kwta.evaluate(X_adv_kwta, y_test)

[5.755090175628662, 0.1309]